In [59]:
import json

import numpy as np


gt = [{"label": "EMAIL_ADDRESS", "text": "d.harrington+alerts@proton.me"}]
lb = [{"label": "EMAIL_ADDRESS", "text": "d.harrington+alerts@proton.me"}]

labels = {item['label'] for item in lb}
labels

{'EMAIL_ADDRESS'}

In [8]:
{ label['label']:dict for label in lb}


{'EMAIL_ADDRESS': dict}

{ label['k']:dict for label in lb}

In [2]:
{k:v for k,v in lb}

In [5]:
ex.get('E')

'blue'

In [19]:

from typing import Literal, TypedDict

Label = [
    "PASSWORD",
    "PERSON",
    "NAME",
    "LOCATION",
    "DATE_OF_BIRTH",
    "NINO",
    "NHS_NUMBER",
    "NATIONAL_INSURANCE_NUMBER",
    "PHONE_NUMBER",
    "BANK_SORT_CODE",
    "BANK_ACCOUNT_NUMBER",
    "CREDIT_CARD_NUMBER",
    "ADDRESS",
    "POSTCODE",
    "VEHICLE_REGISTRATION",
    "EMAIL_ADDRESS",
]

labels = {label for label in Label}

In [20]:
labels

{'ADDRESS',
 'BANK_ACCOUNT_NUMBER',
 'BANK_SORT_CODE',
 'CREDIT_CARD_NUMBER',
 'DATE_OF_BIRTH',
 'EMAIL_ADDRESS',
 'LOCATION',
 'NAME',
 'NATIONAL_INSURANCE_NUMBER',
 'NHS_NUMBER',
 'NINO',
 'PASSWORD',
 'PERSON',
 'PHONE_NUMBER',
 'POSTCODE',
 'VEHICLE_REGISTRATION'}

In [55]:
base_label_dict = {label: {} for label in labels}
base_label_dict

{'PASSWORD': {},
 'POSTCODE': {},
 'NHS_NUMBER': {},
 'PERSON': {},
 'BANK_ACCOUNT_NUMBER': {},
 'CREDIT_CARD_NUMBER': {},
 'NINO': {},
 'LOCATION': {},
 'NATIONAL_INSURANCE_NUMBER': {},
 'BANK_SORT_CODE': {},
 'ADDRESS': {},
 'VEHICLE_REGISTRATION': {},
 'NAME': {},
 'EMAIL_ADDRESS': {},
 'DATE_OF_BIRTH': {},
 'PHONE_NUMBER': {}}

In [56]:
type(base_label_dict['PERSON'])

dict

In [183]:
from typing import List, TypedDict, Optional
import copy
import numpy as np

class Label(TypedDict):
    label: str
    text: str
    score: Optional[float]

LABEL_SET  = {"PASSWORD",
    "PERSON",
    "NAME",
    "LOCATION",
    "DATE_OF_BIRTH",
    "NINO",
    "NHS_NUMBER",
    "NATIONAL_INSURANCE_NUMBER",
    "PHONE_NUMBER",
    "BANK_SORT_CODE",
    "BANK_ACCOUNT_NUMBER",
    "CREDIT_CARD_NUMBER",
    "ADDRESS",
    "POSTCODE",
    "VEHICLE_REGISTRATION",
    "EMAIL_ADDRESS",}

class LabelDictionary(TypedDict):
    PASSWORD: str
    PERSON: str
    NAME: str
    LOCATION: str
    DATE_OF_BIRTH: str
    NINO: str
    NHS_NUMBER: str
    NATIONAL_INSURANCE_NUMBER: str
    PHONE_NUMBER: str
    BANK_SORT_CODE: str
    BANK_ACCOUNT_NUMBER: str
    CREDIT_CARD_NUMBER: str
    ADDRESS: str
    POSTCODE: str
    VEHICLE_REGISTRATION: str
    EMAIL_ADDRESS: str

label_dictionary: LabelDictionary = {
    'PASSWORD': 'PASSWORD',
    'PERSON': 'PERSON',
    'NAME': 'NAME',
    'LOCATION': 'LOCATION',
    'DATE_OF_BIRTH': 'DATE_OF_BIRTH',
    'NINO': 'NINO',
    'NHS_NUMBER': 'NHS_NUMBER',
    'NATIONAL_INSURANCE_NUMBER': 'NATIONAL_INSURANCE_NUMBER',
    'PHONE_NUMBER': 'PHONE_NUMBER',
    'BANK_SORT_CODE': 'BANK_SORT_CODE',
    'BANK_ACCOUNT_NUMBER': 'BANK_ACCOUNT_NUMBER',
    'CREDIT_CARD_NUMBER': 'CREDIT_CARD_NUMBER',
    'ADDRESS': 'ADDRESS',
    'POSTCODE': 'POSTCODE',
    'VEHICLE_REGISTRATION': 'VEHICLE_REGISTRATION',
    'EMAIL_ADDRESS': 'EMAIL_ADDRESS',
}

from typing import List, TypedDict, Optional
import copy
import numpy as np

class Label(TypedDict):
    label: str
    text: str
    score: Optional[float]

LABEL_SET  = {"PASSWORD",
    "PERSON",
    "NAME",
    "LOCATION",
    "DATE_OF_BIRTH",
    "NINO",
    "NHS_NUMBER",
    "NATIONAL_INSURANCE_NUMBER",
    "PHONE_NUMBER",
    "BANK_SORT_CODE",
    "BANK_ACCOUNT_NUMBER",
    "CREDIT_CARD_NUMBER",
    "ADDRESS",
    "POSTCODE",
    "VEHICLE_REGISTRATION",
    "EMAIL_ADDRESS",}

label_dict = {label: {} for label in sorted(LABEL_SET)}
label_dict_set = {label: set() for label in sorted(LABEL_SET)}

class BaseLabels:
    _items: List[Label]
    _labels: set[str]

    def __init__(self, items: List[Label]):
        self._items = items
        self._labels = { label['label'] for label in items }

    @property
    def labels(self) -> set[str]:
        return self._labels

    def __len__(self):
        return len(self._items)

    @property
    def items(self) -> List[Label]:
        return self._items

class Y(BaseLabels):
    _tree: dict[str, set[str]] # TODO: note this doesn't account for multiple entities per label

    def __init__(self, items: List[Label]):
        super().__init__(items)
        self._items = items
        self._tree = copy.deepcopy(label_dict_set)

        for label in items:
            self._tree[label['label']].add(label['text'])

class Predictions(BaseLabels):
    _pred_tree: dict[str, dict[str, float]]
    threshold: float = 0.5
    pred_to_base_labels: dict[str, str]
    _dict: dict[str, str]

    def __init__(self, predictions: List[Label], dictionary: dict[str, str] = label_dictionary):
        super().__init__(predictions)
        self._pred_tree = {k:{} for k in dictionary.values()}
        self.pred_to_base_labels = {v:k for k,v in dictionary.items()}
        self._dict = dictionary
        for label in predictions:
            self._pred_tree[label['label']][label['text']] = (label['score'] if 'score' in label else 1)

    def __len__(self):
        return len(self._items)

    def has_entity(self, gt_label: Label, translation_dict: dict[str,str]) -> bool:
        translated_label = self._dict[gt_label['label']]
        if gt_label['text'] not in self._pred_tree[translated_label]:
            return False

        score = self._pred_tree[translated_label][gt_label['text']]
        return score >= self.threshold

    def has_entity_np(self, label: Label, translation_dict: dict[str,str]) -> np.int64:
        return np.int64(self.has_entity(label, translation_dict))


class Evaluator(Y):
    confusion_matrix: np.ndarray = np.zeros(3).astype(np.int64)
    _to_base_labels: dict[str, str]
    _label_dictionary: dict[str, str]

    def __init__(self, items: List[Label], dictionary: LabelDictionary = label_dictionary):
        super().__init__(items)
        self._to_base_labels = {v:k for k,v in dictionary.items()}
        self._label_dictionary = dict(dictionary)

    def predict(self, predictions_list: List[Label]):
        predictions = Predictions(predictions_list, self._label_dictionary)

        tp = np.array([predictions.has_entity(gt_entity, self._label_dictionary) for gt_entity in self._items]).astype(np.int64)
        fp = np.max(np.int64(0), len(predictions) - tp.sum())
        fn = (1 - tp).sum()

        self.confusion_matrix = np.array([tp.sum(), fp.sum(), fn.sum()]).astype(np.int64)

        return self

    def metrics(self):
        return self.calculate_metrics(self.confusion_matrix)

    @classmethod
    def calculate_metrics(cls, confusion_matrix: np.ndarray):
        tp, fp, fn = confusion_matrix
        accuracy = tp.sum() / (tp.sum() + fp.sum() + fn.sum()) if tp.sum() > 0 else np.float64(0)
        precision = tp.sum() / tp.sum() + fp.sum() if tp.sum() > 0 else np.float64(0)
        recall = tp.sum() / (tp.sum() + fn.sum()) if tp.sum() > 0 else np.float64(0)
        f1 = 2 * ((precision * recall) / (precision + recall)) if tp.sum() > 0 else np.float64(0)

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }



gt: List[Label] = [{"label": "EMAIL_ADDRESS", "text": "d.harrington+alerts@proton.me"}, {"label": "PHONE_NUMBER", "text": "1234"}]
lb: List[Label] = [{"label": "EMAIL_ADDRESS", "text": "d.harrington+alerts@proton.me", "score": 0.9}]
evaluator = Evaluator(gt)

evaluator.predict(lb).confusion_matrix

array([1, 0, 1])

In [191]:
import pandas as pd


class AggEvaluator:
    _base_truth_predictors: List[Evaluator]
    _metrics: List[dict]
    _pred_df: pd.DataFrame

    def __init__(self, base_truth_labels: List[List[Label]]):
        self._base_truth_predictors = [Evaluator(gt) for gt in base_truth_labels]

    def __len__(self):
        return len(self._base_truth_predictors)

    def __iter__(self):
        for x in self._base_truth_predictors:
            yield x

    def predict(self, predictions: pd.DataFrame) -> pd.DataFrame:
        zipped_preds = zip(predictions['labels'], self)
        predictions['results'] = [eval.predict(pred).confusion_matrix for pred, eval in zipped_preds]

        return predictions['results']

    @classmethod
    def instance(cls, labels_path: str = '../data/labels.jsonl'):
        with open(labels_path) as f:
            logs_df = pd.DataFrame({"labels": [json.loads(line.rstrip("\n")) for line in f]})

            return cls(logs_df['labels'])

import json
with open('../data/labels.jsonl') as f:
    logs_df = pd.DataFrame({"labels": [json.loads(line.rstrip("\n")) for line in f]})

# zipped = zip(logs_df['labels'], AggEvaluator(logs_df['labels']))
# logs_df['results'] = [eval.predict(pred).confusion_matrix for pred, eval in zipped]
# logs_df
agg_eval = AggEvaluator.instance()
Evaluator.calculate_metrics(agg_eval.predict(logs_df).sum())

{'accuracy': np.float64(1.0),
 'precision': np.float64(1.0),
 'recall': np.float64(1.0),
 'f1': np.float64(1.0)}

In [174]:
import json
with open('../data/labels.jsonl') as f:
    logs_df = pd.DataFrame({"labels": [json.loads(line.rstrip("\n")) for line in f]})

evaluator = Evaluator(gt)

logs_df['results'] = [evaluator.predict(label).confusion_matrix for label in logs_df['labels']]
logs_df

[{'label': 'EMAIL_ADDRESS', 'text': 'd.harrington+alerts@proton.me'}]
[{'label': 'PHONE_NUMBER', 'text': '+44 (0)20 7946 0958'}, {'label': 'EMAIL_ADDRESS', 'text': 'jane.smith+alerts@acme-mail.co.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'm.ryan+alerts@coastmail.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'a.johnson+alerts@pioneer-mail.co.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'm.wood+alerts@northmail.co.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'amaya.chen+alerts@northwindmail.co.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'm.chen+alerts@inboxmail.co.uk'}]
[{'label': 'PERSON', 'text': 'M. Iqbal'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'devops.alerts+eu-west@acme-example.co.uk'}]
[{'label': 'NHS_NUMBER', 'text': '943 476 5911'}, {'label': 'EMAIL_ADDRESS', 'text': 'jada.rosen+portal@nhs-mail.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'j.sandhu+alerts@northbridge-mail.co.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'sarah.j.holt+billing@outlook.co.uk'}]
[{'label': 'EMAIL_ADDRESS', 'text': 'jane

,labels,results
0,"[{'label': 'EMAIL_ADDRESS', 'text': 'd.harring...","[1, 0, 0]"
1,"[{'label': 'PHONE_NUMBER', 'text': '+44 (0)20 ...","[0, 2, 1]"
2,"[{'label': 'EMAIL_ADDRESS', 'text': 'm.ryan+al...","[0, 1, 1]"
3,"[{'label': 'EMAIL_ADDRESS', 'text': 'a.johnson...","[0, 1, 1]"
4,"[{'label': 'EMAIL_ADDRESS', 'text': 'm.wood+al...","[0, 1, 1]"
...,...,...
312,[],"[0, 0, 1]"
313,"[{'label': 'EMAIL_ADDRESS', 'text': 'm.singh+a...","[0, 1, 1]"
314,"[{'label': 'EMAIL_ADDRESS', 'text': 'maria.gar...","[0, 1, 1]"
315,"[{'label': 'PASSWORD', 'text': 'p@ssW0rd!2026'}]","[0, 1, 1]"


In [104]:
import json
import pandas as pd

with open('../data/labels.jsonl') as f:
    logs_df = pd.DataFrame({"labels": [json.loads(line.rstrip("\n")) for line in f]})

def evaluate(predictions: List[List[Label]]):
    pred_df = pd.DataFrame({"predictions": predictions})


,text
0,"{""timestamp"": ""2026-01-25T14:18:42.917000Z"", ""..."
1,"{""timestamp"": ""2026-01-25T14:18:32.947000Z"", ""..."
2,"{""timestamp"": ""2026-01-25T16:41:12.904000Z"", ""..."
3,"{""timestamp"": ""2026-01-25T14:12:08.443000Z"", ""..."
4,"{""timestamp"": ""2026-01-25T10:18:42.317000Z"", ""..."
...,...
312,"{""timestamp"": ""2026-01-25T10:18:42.771000Z"", ""..."
313,"{""timestamp"": ""2026-01-25T11:42:18.903000Z"", ""..."
314,"{""timestamp"": ""2026-01-25T09:42:18.317000Z"", ""..."
315,"{""timestamp"": ""2026-01-25T09:41:12.684000Z"", ""..."


In [93]:
sum([np.array([1,2,3]),np.array([1,2,3]),np.array([1,2,3])])

array([3, 6, 9])

In [196]:
MAX_CHARS = 10
l = []
paragraph = "This is a long sentence that will be split into chunks of 5 characters.\nThis is a second line."
for line in paragraph.splitlines():
    l.append([line[i:i + MAX_CHARS] for i in range(0, len(line), MAX_CHARS)])

l

[['This is a ',
  'long sente',
  'nce that w',
  'ill be spl',
  'it into ch',
  'unks of 5 ',
  'characters',
  '.'],
 ['This is a ', 'second lin', 'e.']]

In [197]:
line = ['a','b','c','d','e']

for chunk in line:
    chunk = 1

line

['a', 'b', 'c', 'd', 'e']

In [200]:
import time

start = time.perf_counter()
time.perf_counter() - start

1.2040953151881695e-05